<a href="https://colab.research.google.com/github/r7him/Abderrahmen0102/blob/main/Suppliers_warehouse_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1797]:
!pip install gurobipy >10
from gurobipy import *
import numpy as np 
import pandas as np

In [1798]:
# Warehouse demand in thousands of units
demand = [20, 10]
waste= [15, 30]

In [1799]:
# supplier capacity in thousands of units
CS = [20, 15, 13,12]
#warehouse capacity
CW = [18, 22, 15]
#warehouse capacity
CG = [200, 100,50]
CO = [200, 100]

In [1800]:
# Fixed costs for selectin a supplier.
Fs = [2000, 15000, 17000,10000]

In [1801]:
# Fixed costs for each warehouses.
Fw = [16000, 15000,25000]

In [1802]:
# Fixed costs for each storage.
Fg = [12000, 10000,17000]

In [1803]:
# Fixed costs for each storage.
#Fo = [12000, 5000]

In [1804]:
# Transportation costs from suppliers to warehouses
TC_sw =       [[4000, 2000, 3000, 2500],
              [2500, 2600, 3400, 3000],
              [1200, 1800, 2600, 4100]]
# Transportation costs from warehouses to healthcare centres

TC_wh =      [[200, 150, 120],
              [250, 260,130]]

# Transportation costs of vaccine waste from to healthcare centres to waste storage.

TC_hg=       [[4000, 2000],
              [2500, 2600],
              [1200, 1800]]
# Transportation costs of vaccine waste from waste storage treatment storage.
TC_go=       [[4000, 2000, 1500],
              [2500, 2600, 7000]]

In [1805]:
# Range of plants and warehouses
suppliers = range(len(CS))
warehouses = range(len(CW))
healthcentre= range(len(demand))
storages    = range(len(CG))
treatments = range(len(waste))

In [1806]:
m=Model()

In [1807]:
#suppliers open decision variables: S_s[s] == 1 if supplier s is open.
S_s = m.addVars(suppliers,
                 vtype=GRB.BINARY,
                 obj=Fs,
                 name="S_s")
# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
# S_s  = []
# for s in suppliers:
#     S_s .append(m.addVar(vtype=GRB.BINARY,
#                          obj=fixedCost12[s],
#                          name="S_s[%d]" % s))

In [1808]:
#Warehous open decision variables: W_w[w] == 1 if warehouse w is open.
W_w = m.addVars(warehouses,
                 vtype=GRB.BINARY,
                 obj=Fw,
                 name="W_w")
# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
#W_w   = []
# for w in warehouses:
#     W_w .append(m.addVar(vtype=GRB.BINARY,
#                          obj=fixedCost23[w],
#                          name="W_w[%d]" % w))

In [1809]:
G_g = m.addVars(storages,
                vtype=GRB.BINARY, 
                obj=Fg , 
                name="G_g")

In [1810]:
#O_o = m.addVars(treatments,
                #vtype=GRB.BINARY, 
                #obj=Fo , 
                #name="O_o")

In [1811]:
# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
(QS_sw)= m.addVars(warehouses,suppliers,obj=TC_sw, name="QS_sw")
# QS_sw = []
# for s in suppliers:
#     QS_sw.append([])
#     for s in suppliers:
#         QS_sw[w].append(m.addVar(obj=transCost12[w][p],
#                                      name="trans[%d,%d]" % (w, s)))

In [1812]:
# QW_w decision variables: QW_wh[w,p] captures the
# optimal quantity to transport to warehouse w from supplier s
QW_wh= m.addVars(healthcentre,warehouses, obj=TC_wh, name="QW_wh")


In [1813]:
# QG_g decision variables: QW_wh[w,p] captures the
# optimal quantity to transport from healtcare to storages.
QG_hg= m.addVars(healthcentre,storages, obj=TC_hg, name="QG_hg")

In [1814]:
QO_go= m.addVars(storages,treatments, obj=TC_go, name="QO_go")

In [1815]:
# QW_w decision variables: QW_wh[w,p] captures the
# optimal quantity to transport to warehouse w from supplier s
#QU_wh= m.addVar(vtype=GRB.CONTINUOUS, name="QU_wh")

In [1816]:
#m.setObjective((TC_sw[s,w] for s in TC_sw for w in TC_sw*QS_sw[w] for w in QS_sw), GRB.MINIMIZE)
m.ModelSense=GRB.MINIMIZE


In [1817]:
# Production constraints # Note that the right-hand limit sets the production to zero if the plant # is closed
m.addConstrs(((QS_sw).sum('*', s) <= CS[s]*S_s[s] for s in suppliers), "Capacity of suppliers")
# Using Python looping constructs, the preceding would be...
# for s in suppliers:
#     m.addConstr(sum(QS_sw[w][s] for w in warehouses)
#                 <= capacity[s] * S_s[s], "Capacity[%d]" % s)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [1818]:
# warehouses capacity constraints
m.addConstrs(((QW_wh).sum('*',w) <= CW[w]*W_w[w] for w in warehouses ), "Capacity of warehouses")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [1819]:
# storges capacity constraints
m.addConstrs(((QG_hg).sum('*',g) <= CG[g]*G_g[g] for g in storages ), "Capacity of storages")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [1820]:
# treatment capacity constraints
#m.addConstrs(((QO_go).sum('*',o) <= CO[o]*O_o[o] for o in treatments ), "Capacity of treatments")

In [1821]:
# Demand constraints
m.addConstrs(
    (QW_wh.sum(h,'*') >= demand[h] for h in healthcentre), "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [1822]:
# waste constraints
m.addConstrs(
   ( QO_go.sum(o,'*') == waste[o] for o in treatments), "waste")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [1823]:
#Flow Balance constraint between suppliers and warehouses
m.addConstrs((QS_sw.sum(w,'*') == QW_wh.sum('*',w) for w in warehouses), "flow balance")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [1824]:
#Flow Balance constraint between suppliers and warehouses
m.addConstrs((QO_go.sum(o,'*') == QG_hg.sum('*',o) for o in treatments), "flow balance")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [1825]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 19 rows, 40 columns and 70 nonzeros
Model fingerprint: 0xd30c46fc
Variable types: 30 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 7 rows and 15 columns
Presolve time: 0.00s
Presolved: 12 rows, 25 columns, 49 nonzeros
Variable types: 18 continuous, 7 integer (7 binary)
Found heuristic solution: objective 282900.00000

Root relaxation: objective 2.764480e+05, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 276447.980    0    3 282900.

In [1826]:
for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % m.ObjVal)

S_s[0] 1
S_s[1] 1
S_s[2] 0
S_s[3] 0
W_w[0] 1
W_w[1] 0
W_w[2] 1
G_g[0] 1
G_g[1] 1
G_g[2] 0
QS_sw[0,0] 0
QS_sw[0,1] 15
QS_sw[0,2] 0
QS_sw[0,3] 0
QS_sw[1,0] 0
QS_sw[1,1] 0
QS_sw[1,2] 0
QS_sw[1,3] 0
QS_sw[2,0] 15
QS_sw[2,1] 0
QS_sw[2,2] 0
QS_sw[2,3] 0
QW_wh[0,0] 15
QW_wh[0,1] 0
QW_wh[0,2] 5
QW_wh[1,0] 0
QW_wh[1,1] 0
QW_wh[1,2] 10
QG_hg[0,0] 0
QG_hg[0,1] 0
QG_hg[0,2] 0
QG_hg[1,0] 15
QG_hg[1,1] 30
QG_hg[1,2] 0
QO_go[0,0] 0
QO_go[0,1] 15
QO_go[1,0] 30
QO_go[1,1] 0
QO_go[2,0] 0
QO_go[2,1] 0
Obj: 282900
